In [1]:
import glob
import os
import datetime
import pandas as pd
import numpy as np
import gc
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import lightgbm as lgb

In [2]:
# 指定したファイルパスのcsvファイルを全部読み込む関数
def read_csv_all(target_path = '../input/'):
    for file in glob.glob(target_path+'*.csv'):
        raw_name = 'raw_' + file.replace(target_path,'').replace('.csv','')
        exec('{} = pd.read_csv(\'{}\')'.format(raw_name, file), globals())
        print('read {} as {}'.format(file, raw_name))

In [3]:
def reduce_mem_usage(df, verbose=False):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    int_columns = df.select_dtypes(include=["int"]).columns
    float_columns = df.select_dtypes(include=["float"]).columns

    for col in int_columns:
        df[col] = pd.to_numeric(df[col], downcast="integer")

    for col in float_columns:
        df[col] = pd.to_numeric(df[col], downcast="float")

    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [4]:
read_csv_all()

read ../input/calendar.csv as raw_calendar
read ../input/sell_prices.csv as raw_sell_prices
read ../input/sales_train_validation.csv as raw_sales_train_validation
read ../input/sample_submission.csv as raw_sample_submission


In [5]:
NUM_ITEMS = raw_sales_train_validation.shape[0]  # 30490
DAYS_PRED = raw_sample_submission.shape[1] - 1  # 28

In [6]:
def encode_categorical(df, cols):
    for col in cols:
        # Leave NaN as it is.
        le = LabelEncoder()
        not_null = df[col][df[col].notnull()]
        df[col] = pd.Series(le.fit_transform(not_null), index=not_null.index)

    return df

In [7]:
calendar = encode_categorical(
    raw_calendar, ["event_name_1", "event_type_1", "event_name_2", "event_type_2"]
).pipe(reduce_mem_usage)

sales = encode_categorical(
    raw_sales_train_validation, ["item_id", "dept_id", "cat_id", "store_id", "state_id"],
).pipe(reduce_mem_usage)
del raw_sales_train_validation

prices = encode_categorical(raw_sell_prices, ["item_id", "store_id"]).pipe(reduce_mem_usage)

In [8]:
submission = raw_sample_submission
del raw_sample_submission

In [9]:
sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,1438,3,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,1439,3,1,0,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,1440,3,1,0,0,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,1441,3,1,0,0,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [10]:
submission.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
def extract_num(ser):
    return ser.str.extract(r"(\d+)").astype(np.int16)

In [12]:
# salesをDMの形に変換
id_columns = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"]

# get product table.
product = sales[id_columns]

sales = sales.melt(id_vars=id_columns, var_name="d", value_name="demand",)
sales = reduce_mem_usage(sales)

In [13]:
# submissionをDMの形に変換

# separate test dataframes.
vals = submission[submission["id"].str.endswith("validation")]
evals = submission[submission["id"].str.endswith("evaluation")]

# change column names.
vals.columns = ["id"] + [f"d_{d}" for d in range(1914, 1914 + DAYS_PRED)]
evals.columns = ["id"] + [f"d_{d}" for d in range(1942, 1942 + DAYS_PRED)]

In [14]:
# merge with product table
evals["id"] = evals["id"].str.replace("_evaluation", "_validation")
vals = vals.merge(product, how="left", on="id")
evals = evals.merge(product, how="left", on="id")
evals["id"] = evals["id"].str.replace("_validation", "_evaluation")

vals = reduce_mem_usage(vals)
evals = reduce_mem_usage(evals)

/Users/hojokenta/.local/share/virtualenvs/M5_Forecasting_Accuracy-hifQUVP_/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
# valsとevalsをidで縦持ちに変換
vals = vals.melt(id_vars=id_columns, var_name="d", value_name="demand")
evals = evals.melt(id_vars=id_columns, var_name="d", value_name="demand")

In [16]:
sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand
0,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,d_1,0
1,HOBBIES_1_002_CA_1_validation,1438,3,1,0,0,d_1,0
2,HOBBIES_1_003_CA_1_validation,1439,3,1,0,0,d_1,0
3,HOBBIES_1_004_CA_1_validation,1440,3,1,0,0,d_1,0
4,HOBBIES_1_005_CA_1_validation,1441,3,1,0,0,d_1,0


In [17]:
# sales,vals,evalsをガッチャンコ
sales["part"] = "train"
vals["part"] = "validation"
evals["part"] = "evaluation"
data = pd.concat([sales, vals, evals], axis=0)
del sales, vals, evals

In [18]:
data["d"] = extract_num(data["d"])

In [19]:
# 期間を適切な期間で絞る
# 時間でcvするためコメントアウト
# d_thresh = 1941 - int(365*2)
# data = data[data["d"] >= d_thresh]
# 今はevalを削除
data = data[data["part"] != "evaluation"]
gc.collect()

32

In [20]:
def merge_calendar(data, calendar):
    calendar = calendar.drop(["weekday", "wday", "month", "year"], axis=1)
    return data.merge(calendar, how="left", on="d")


def merge_prices(data, prices):
    return data.merge(prices, how="left", on=["store_id", "item_id", "wm_yr_wk"])

In [21]:
gc.collect()

40

In [22]:
# カレンダーと商品価格をマージ
calendar["d"] = extract_num(calendar["d"])
data = merge_calendar(data, calendar)
del calendar
gc.collect()

data = merge_prices(data, prices)
del prices
gc.collect()

data = reduce_mem_usage(data)

In [23]:
# dataをtrainに絞って学習してみる
train = data[data["part"] == "train"]
test = data[data["part"] == "validation"]

In [24]:
# 目的変数と特徴量に分解
features = ["item_id", "dept_id", "cat_id", "store_id", "d", "wm_yr_wk", "event_name_1", "event_type_1", "event_name_2", "event_type_2", "snap_CA", "snap_TX", "snap_WI", "sell_price"]
train_y = train["demand"]
train_x = train[features]
test_x = test[features]

In [25]:
# submit用に加工するためにidとdateをとっておく
id_date = data[data["part"] != "train"][["id", "date"]].reset_index(drop=True)

In [26]:
train.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,1,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN
1,HOBBIES_1_002_CA_1_validation,1438,3,1,0,0,1,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN
2,HOBBIES_1_003_CA_1_validation,1439,3,1,0,0,1,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN
3,HOBBIES_1_004_CA_1_validation,1440,3,1,0,0,1,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN
4,HOBBIES_1_005_CA_1_validation,1441,3,1,0,0,1,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN


In [27]:
train['date'] = pd.to_datetime(train['date'])
max_date = max(train['date'])
valid_period = 28
train_period = 365 * 2

/Users/hojokenta/.local/share/virtualenvs/M5_Forecasting_Accuracy-hifQUVP_/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [28]:
max_date - datetime.timedelta(days=valid_period-1)

Timestamp('2016-03-28 00:00:00')

In [29]:
va = train[train['date'] >= max_date - datetime.timedelta(days=valid_period-1)]
tr = train[(train['date'] <= max_date - datetime.timedelta(days=valid_period)) & (train['date'] >= max_date - datetime.timedelta(days=valid_period+train_period-1))]
tr_y = tr['demand']
tr_x = tr[features]
va_y = va['demand']
va_x = va[features]

In [31]:
# lightgbmで時系列CVしてみる

## lightgbmの構造に変換
lgb_train = lgb.Dataset(tr_x, tr_y)
lgb_eval = lgb.Dataset(va_x, va_y)

## ハイパーパラメータの設定
params = {'objective': 'regression', 'seed': 71, 'verbose': 0, 'metrics': 'rmse'}
num_round = 100

## カテゴリカル変数の指定
categorical_features = ["item_id", "dept_id", "cat_id", "store_id", "event_name_1", "event_type_1", "event_name_2", "event_type_2"]

model = lgb.train(params, lgb_train, num_boost_round=num_round,
                  categorical_feature=categorical_features,
                  valid_names=['tarin', 'valid'], valid_sets=[lgb_train, lgb_eval])

/Users/hojokenta/.local/share/virtualenvs/M5_Forecasting_Accuracy-hifQUVP_/lib/python3.7/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat_id', 'dept_id', 'event_name_1', 'event_name_2', 'event_type_1', 'event_type_2', 'item_id', 'store_id']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[1]	tarin's rmse: 3.45813	valid's rmse: 3.4404
[2]	tarin's rmse: 3.32611	valid's rmse: 3.30856
[3]	tarin's rmse: 3.21491	valid's rmse: 3.19718
[4]	tarin's rmse: 3.11936	valid's rmse: 3.106
[5]	tarin's rmse: 3.03917	valid's rmse: 3.02536
[6]	tarin's rmse: 2.97185	valid's rmse: 2.95742
[7]	tarin's rmse: 2.91545	valid's rmse: 2.90032
[8]	tarin's rmse: 2.86768	valid's rmse: 2.85474
[9]	tarin's rmse: 2.82625	valid's rmse: 2.8159
[10]	tarin's rmse: 2.7927	valid's rmse: 2.78175
[11]	tarin's rmse: 2.7602	valid's rmse: 2.75119
[12]	tarin's rmse: 2.73469	valid's rmse: 2.72594
[13]	tarin's rmse: 2.71262	valid's rmse: 2.70329
[14]	tarin's rmse: 2.69222	valid's rmse: 2.68395
[15]	tarin's rmse: 2.67496	valid's rmse: 2.66642
[16]	tarin's rmse: 2.65882	valid's rmse: 2.65291
[17]	tarin's rmse: 2.6425	valid's rmse: 2.63954
[18]	tarin's rmse: 2.63146	valid's rmse: 2.62897
[19]	tarin's rmse: 2.62077	valid's rmse: 2.61979
[20]	tarin's rmse: 2.61096	valid's rmse: 2.60877
[21]	tarin's rmse: 2.59835	valid's r

In [32]:
preds = model.predict(test_x)

In [33]:
def make_submission(test, submission):
    preds = test[["id", "date", "demand"]]
    preds = preds.pivot(index="id", columns="date", values="demand").reset_index()
    preds.columns = ["id"] + ["F" + str(d + 1) for d in range(DAYS_PRED)]

    vals = submission[["id"]].merge(preds, how="inner", on="id")
    evals = submission[submission["id"].str.endswith("evaluation")]
    final = pd.concat([vals, evals])

    assert final.drop("id", axis=1).isnull().sum().sum() == 0
    assert final["id"].equals(submission["id"])

    return final

In [34]:
submit_file = make_submission(id_date.assign(demand=preds), submission)
submit_file.to_csv("../output/submission_baseline_timevalid_re.csv", index=False)